In [1]:
import html
import re

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import tensorflow_hub as hub
import tensorflow as tf

import keras

C:\Users\ausaf\Anaconda3\envs\tensorflow_environment\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

INFO:tensorflow:Using C:\Users\ausaf\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'C:\\Users\\ausaf\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with aggregation/scaling
INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'C:\\Users\\ausaf\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with aggregation/weights
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_0:0 from checkpoint b'C:\\Users\\ausaf\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/CNN/W_cnn_0
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_1:0 from checkpoint b'C:\\Users\\ausaf\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/CNN/W_cnn_1
INFO:tensorflow:

INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel:0 from checkpoint b'C:\\Users\\ausaf\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias:0 from checkpoint b'C:\\Users\\ausaf\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'C:\\Users\\ausaf\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/

In [5]:
sess = tf.InteractiveSession()

C:\Users\ausaf\Anaconda3\envs\tensorflow_environment\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [6]:
sess.run(tf.global_variables_initializer())

In [7]:
embeddings = elmo_model(
    ["the cat is on the mat", "dogs are in the fog"],
    signature="default",
    as_dict=True
)["elmo"]

In [8]:
e = sess.run(embeddings)
e.shape

(2, 6, 1024)

In [9]:
whitespace_re = re.compile('\s+')
word_re = re.compile('\w+')

In [10]:
x = list()
y = list()
with open('train.dat', 'r') as fr:
    for i, line in enumerate(fr):
        if i == 0:
            continue
        classification, line = whitespace_re.split(line, maxsplit=1)
        line = line.strip()
        line = html.unescape(line)
        line = ' '.join(word_re.findall(line))
        x.append(line)
        y.append(classification)

In [11]:
y = list(map(int, y))
lb = LabelBinarizer()
lb.fit(y)
y = lb.transform(y)

In [12]:
num_classes = y.shape[-1]

In [13]:
y[:10]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [14]:
x[:10]

['she had another song out recently but it didn t get that much play',
 'wellity i think ima gonna clean my room Gay i need to read a good book sense and sensibility here we come BEAT THE LILY',
 'Bug in damn cod4 system link My map pack isnt working',
 'weeooow i feel fat i remember when i used to weigh that much pfffffffffffffffffffffftttttttt long time ago hahaha',
 'mmmm now you re speaking my language Unfortunately in militant diet mode am seriously thinking of competiton so no TB I guess',
 'Really I feel special',
 'don t you procrastinate like I do',
 'Ive been wanting 2 but will be gone 4 2 mo have to close my shops',
 'Sushi two nights in a row isn t THAT wrong is it',
 'haha i get my senior license next friday']

In [15]:
max_sentence_length = 0
for line in x:
    max_sentence_length = max(max_sentence_length, len(line.split()))
max_sentence_length

50

In [16]:
embeddings = elmo_model(
    x[:2],
    signature="default",
    as_dict=True
)["elmo"]

In [17]:
%%time
e = sess.run(embeddings)

Wall time: 2.01 s


In [18]:
e.shape

(2, 25, 1024)

In [19]:
embedding_vector_length = e.shape[-1]

In [20]:
e.shape

(2, 25, 1024)

In [21]:
# https://keras.io/layers/core/
# https://keras.io/getting-started/sequential-model-guide/
model = keras.Sequential()
model.add(keras.layers.LSTM(
    100,
    input_shape=(max_sentence_length,embedding_vector_length),
))
model.add(keras.layers.Dense(
    num_classes,
    # Hands-On Machine Learning with Scikit-Learn and TensorFlow, ed. 1 by Aurélien Géron
    activation='softmax',
))
model.compile(
    # https://www.liip.ch/en/blog/sentiment-detection-with-keras-word-embeddings-and-lstm-deep-learning-networks
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               450000    
_________________________________________________________________
dense_1 (Dense)              (None, 11)                1111      
Total params: 451,111
Trainable params: 451,111
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
def pad(e, sentence_length=max_sentence_length):
    # https://stackoverflow.com/questions/35751306/python-how-to-pad-numpy-array-with-zeros
    num_sentences, old_sentence_length, embedding_length = e.shape
    e2 = np.zeros((num_sentences, sentence_length, embedding_length))
    e2[:, :old_sentence_length, :] = e
    return e2

In [23]:
e2 = pad(e)

In [24]:
model.fit(e2, y[:2])

Epoch 1/1
2/2 [==============================] - 1s 284ms/step - loss: 2.3967 - acc: 0.5000


In [25]:
pred = model.predict(e2)

In [26]:
pred.argmax(axis=1) == y[:2].argmax(axis=1)

array([ True,  True])